# Block 6: Feature Selection

## Objectives

- Enumerate all combinations of features
- Implement and test brute force selection
- Implement and test forward greedy selection
- Use brute force and greedy within the proper nested-CV setup

In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import sklearn.model_selection
import itertools
from typing import List

In [2]:
#
# Copied these functions from block4 and block5
#

def prepare_inputs(df):
    # we need to separate categorical from numeric features
    # because they require separate processing
    # let's get categorical columns
    categorical_cols = df.select_dtypes(include='object').columns
    
    # let's get numeric
    ordinal_cols = df.select_dtypes(include='number').columns

    # construct input features
    X = df[ordinal_cols].to_numpy()

    # z-score (NxK' - 1xK') / 1xK' = NxK'
    X = (X - np.mean(X, axis=0)[None, :]) / np.std(X, axis=0)[None, :]

    # code categorical features
    for feature in categorical_cols:
        dummies = pd.get_dummies(df[feature]).to_numpy().astype(float)
        X = np.hstack((X, dummies)) 

    # add a column of ones
    ones_col = np.ones((X.shape[0], 1)) # Nx1
    X = np.hstack((ones_col, X)) # K

    return X 

def forward_fn(Beta, X):
    """
        Beta: K
        X: NxK
    """
   
    return X @ Beta # NxK @ K = N

def optimize(df, y, eta, steps):

    X = prepare_inputs(df)
    
    # randomly initialize solution 
    Beta = np.random.rand(X.shape[1]) # K
    
    # iterate for steps
    history = []

    for i in range(steps):
        yhat = forward_fn(Beta, X)
        mse = np.mean(np.square(yhat - y))
        history.append([Beta, mse])

        # compute gradient at those predictions
        # (NxK).T @ N = K
        Beta_grad = 2 * X.T @ (yhat - y) / X.shape[0]
        
        # update solution
        Beta = Beta - eta * Beta_grad
        
    return Beta, history 
def predict(Beta, df):
    X = prepare_inputs(df)
    
    return forward_fn(Beta, X)

#
# this is a function ... that returns a function :)
#
def create_mvlr_train_test_fn(features, output_col):
    
    def train_test_fn(df_train, df_test):
        params, _ = optimize(df_train[features], df_train[output_col], eta = 0.1, steps=100)
        yhat = predict(params, df_test[features])
        ytest = df_test[output_col].to_numpy()
        mse = np.mean(np.square(yhat - ytest))
        return mse 

    return train_test_fn

#
# Cross-validation function
#
def cv(df, train_test_fn, folds, random_state):

    # instantiate the splitter
    kf = sklearn.model_selection.KFold(n_splits=folds, 
                                       shuffle=True, 
                                       random_state=random_state)
    
    metrics = []
    for train_index, test_index in kf.split(df):
        train_df = df.iloc[train_index]
        test_df = df.iloc[test_index]
        
        # evaluate
        metric = train_test_fn(train_df, test_df)
        metrics.append(metric)
    
    return metrics 


## Brute Force Selection

In [3]:
# let's play with combinations function because we'll use it in brute force
# selection
cols = ['a', 'b', 'c']

# combinations of length 3
print(list(itertools.combinations(cols, 1)))
# combinations of length 2
print(list(itertools.combinations(cols, 2)))
# combinations of length 3
print(list(itertools.combinations(cols, 3)))

# to get all combinations
all_combinations = [comb for i in range(1, len(cols)) for comb in itertools.combinations(cols, i)]
print(all_combinations)

[('a',), ('b',), ('c',)]
[('a', 'b'), ('a', 'c'), ('b', 'c')]
[('a', 'b', 'c')]
[('a',), ('b',), ('c',), ('a', 'b'), ('a', 'c'), ('b', 'c')]


In [4]:
# load dataset
df = pd.read_json('../data/cars.json')

# Filter dataframe
required_cols = ['Miles_per_Gallon', 'Cylinders', 'Displacement', 'Horsepower', 'Weight_in_lbs', 'Acceleration', 'Origin']

# only include rows where ALL columns are not nan
ix_included = np.sum(pd.isna(df[required_cols]), axis=1) == 0

# exclude examples with no horsepower or mpg
print("Before: ", df.shape)
df = df[ix_included]
print("After: ", df.shape)


Before:  (406, 9)
After:  (392, 9)


In [8]:
def brute_force_selection(df,
                          input_cols,
                          output_col,
                          create_train_test_fn):

    # generate all combinations
    all_combinations = [comb for i in range(1, len(input_cols)) for comb in itertools.combinations(input_cols, i)]

    # start cross validation FOR EACH combination
    all_metrics = []
    for combination in all_combinations:
        # important to keep random state the same for all combinations
        # so that generated splits are the same
        train_test_fn = create_train_test_fn(list(combination), output_col)

        metrics = cv(df, train_test_fn, folds=5, random_state=23412341)
        all_metrics.append(metrics)
    
    # organize all results in a 2D matrix (Rows = combinations, cols = folds)
    all_metrics = np.array(all_metrics) # Combinations x Folds

    # compute average metric for each combination
    avg_metric = np.mean(all_metrics, axis=1) # Combinations
    
    # pick best
    best_ix = np.argmin(avg_metric)
    best_combination = list(all_combinations[best_ix])
    
    return best_combination


features = ['Cylinders', 'Displacement', 'Horsepower', 'Weight_in_lbs', 'Acceleration', 'Origin']

best_combination = brute_force_selection(df, input_cols=features, output_col='Miles_per_Gallon', create_train_test_fn=create_mvlr_train_test_fn)

best_combination

['Horsepower', 'Weight_in_lbs', 'Origin']

## Forward Greedy Selection

In [6]:
def forward_greedy_selection(df,
                             input_cols,
                             output_col,
                             create_train_test_fn):
    
    
    current_combination = []
    current_metric = np.inf
    rem_features = input_cols

    while len(rem_features) > 0:
        
        all_metrics = []
        for feature in rem_features:
            
            # create candidate
            candidate_combination = current_combination + [feature]

            # important to keep random state the same for all combinations
            # so that generated splits are the same
            train_test_fn = create_train_test_fn(candidate_combination, output_col)

            metrics = cv(df, train_test_fn, folds=5, random_state=23412341)
            all_metrics.append(metrics)
        
        # organize all results in a 2D matrix (Rows = Rem Features, cols = folds)
        all_metrics = np.array(all_metrics) # Rem Features x Folds

        # compute average metric for each combination
        avg_metric = np.mean(all_metrics, axis=1) # Combinations
        
        # pick best
        best_ix = np.argmin(avg_metric)

        best_metric = avg_metric[best_ix]
        if best_metric > current_metric:
            # no combination improved on current best, stop
            break
        else:
            current_metric = best_metric
            best_feature = rem_features[best_ix]
            
            # update
            current_combination = current_combination + [best_feature]

            # remove from remaining features
            rem_features = [f for f in rem_features if f != best_feature]

    return current_combination

best_combination = forward_greedy_selection(df, input_cols=features, output_col='Miles_per_Gallon', create_train_test_fn=create_mvlr_train_test_fn)

best_combination

['Weight_in_lbs', 'Horsepower', 'Origin']

In [9]:
def create_feature_selection_train_test_fn(features, output_col, method='greedy'):

    # pick the strategy for feature selection based on the method parameter
    if method == 'greedy':
        feature_selection_fn = forward_greedy_selection
    elif method == 'brute':
        feature_selection_fn = brute_force_selection
    else:
        raise Exception("Unknown method")
    
    def train_test_fn(df_train, df_test):
        # find best combination
        best_combination = feature_selection_fn(df_train, input_cols=features, output_col=output_col, create_train_test_fn=create_mvlr_train_test_fn)
        print(best_combination)

        # fit the model to the whole training dataset
        best_p, _ = optimize(df_train[best_combination], df_train[output_col].to_numpy(), eta=0.1, steps=100)

        # evaluate it
        yhat = predict(best_p, df_test[best_combination])
        ytest = df_test[output_col].to_numpy()
        mse = np.mean(np.square(yhat - ytest))
        return mse 
    return train_test_fn

train_test_fn = create_feature_selection_train_test_fn(features, 'Miles_per_Gallon', 'brute')
metrics = cv(df, train_test_fn, folds=5, random_state=23424)
np.mean(metrics)

['Horsepower', 'Weight_in_lbs', 'Origin']
['Horsepower', 'Weight_in_lbs', 'Origin']
['Horsepower', 'Weight_in_lbs']
['Horsepower', 'Weight_in_lbs', 'Acceleration', 'Origin']
['Horsepower', 'Weight_in_lbs', 'Origin']


np.float64(18.55799991409059)